In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

In [2]:
from diffusion.data_loaders.spinkick_pos_only_dataset import SpinkickPosOnlyDataset
dataset = SpinkickPosOnlyDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_spinkick.txt")
len(dataset), dataset[0].shape

(1000, torch.Size([78, 35]))

In [3]:
dataset[0]

tensor([[ 0.0000,  0.0000,  0.8251,  ..., -0.5877, -0.5578, -0.1123],
        [-0.0098, -0.0053,  0.8223,  ..., -0.6034, -0.5496, -0.1377],
        [-0.0205, -0.0135,  0.8187,  ..., -0.6073, -0.5372, -0.1672],
        ...,
        [ 0.4499, -0.1250,  0.7884,  ..., -0.6189,  0.1019, -0.4792],
        [ 0.4598, -0.1251,  0.8026,  ..., -0.6567, -0.0243, -0.4099],
        [ 0.4689, -0.1252,  0.8251,  ..., -0.6718, -0.1104, -0.3359]])

### Model NN

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # not used in the final model
        x = x + self.pe[:x.shape[0], :]
        return self.dropout(x)


class TimestepEmbedder(nn.Module):
    def __init__(self, latent_dim, sequence_pos_encoder):
        super().__init__()
        self.latent_dim = latent_dim
        self.sequence_pos_encoder = sequence_pos_encoder

        time_embed_dim = self.latent_dim
        self.time_embed = nn.Sequential(
            nn.Linear(self.latent_dim, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )

    def forward(self, timesteps):
        return self.time_embed(self.sequence_pos_encoder.pe[timesteps])

In [5]:
class MotionTransformer(nn.Module):
    def __init__(self, pos_dim, latent_dim=256, ff_size=1024, num_layers=8, num_heads=4, dropout=0.1, activation="gelu"):
        super(MotionTransformer, self).__init__()
        
        # self.nfeats = nfeats
        self.pos_dim = pos_dim
        self.latent_dim = latent_dim
        self.ff_size = ff_size  
        self.dropout = dropout

        self.posEmbedding = nn.Linear(self.pos_dim, self.latent_dim)
        self.sequence_pos_encoder = PositionalEncoding(self.latent_dim, self.dropout)
        self.embed_timestep = TimestepEmbedder(self.latent_dim, self.sequence_pos_encoder)

        # Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(d_model=self.latent_dim, nhead=num_heads, 
                                                    dim_feedforward=ff_size, dropout=dropout, activation=activation, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Output Linear Layer
        self.posOutEmbedding = nn.Linear(self.latent_dim, self.pos_dim)

    def forward(self, x: torch.Tensor, timesteps, y=None, verbose=False):
        """
        x: [batch_size, max_frames, n_feats], denoted x_t in the paper
        timesteps: [batch_size] (int)
        """
        # x: [batch_size, seq_len, nfeats]
        emb = self.embed_timestep(timesteps)  # [bs, n_frames, time_embed_dim]
        if verbose:
            print("Emb", emb.shape)
            print(emb)

        x = self.posEmbedding(x) # [bs, n_frames, n_dim]
        if verbose:
            print("Frame Embedding", x.shape)
            print(x)

        # Transformer Encoder
        # adding the timestep embed
        xseq = torch.cat((emb, x), axis=1)  # [bs, n_frames+1, n_dim]
        if verbose:
            print("Concat x and zkx", xseq.shape)
            print(xseq)

        xseq = self.sequence_pos_encoder(xseq)  # [bs, n_frames+1, n_dim]
        if verbose:
            print("Sequence Pos Encoder", xseq.shape)
            print(xseq)

        output = self.transformer_encoder(xseq)[:, 1:, :]  # , src_key_padding_mask=~maskseq)  # [bs, n_frames, n_dim]
        if verbose:
            print("Transformer Encoder", output.shape)
            print(output)

        # Output Linear
        output = self.posOutEmbedding(output)
        if verbose:
            print("Output Embedding", output.shape)
            print(output)

        return output

In [18]:
from torch.utils.data import DataLoader

batch_size = 1
dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        num_workers=8, drop_last=True)

pos_dim = dataset[0].shape[1]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MotionTransformer(pos_dim=pos_dim, latent_dim=128, ff_size=512, num_layers=8, num_heads=4, dropout=0.1, activation="gelu").to(device)

pos_dim

35

In [19]:
test_sample = dataset[0].unsqueeze(0).to(device)    
print(test_sample.shape)

res = model(test_sample, torch.tensor([0]).to(device), verbose=True)
res.shape

torch.Size([1, 78, 35])
Emb torch.Size([1, 1, 128])
tensor([[[-5.2326e-02,  1.0404e-01,  1.1141e-01, -3.1471e-01,  6.5154e-02,
          -6.4290e-02,  6.9357e-02,  1.5860e-01,  6.4102e-03,  1.9527e-01,
          -1.3651e-01, -1.1225e-01, -1.5034e-01, -3.7038e-02, -4.7231e-02,
          -1.3426e-01,  2.6856e-02,  7.7447e-02, -1.1523e-02, -2.8833e-02,
          -6.0362e-02, -8.5810e-02,  2.4798e-02, -2.7860e-02, -1.0549e-01,
          -4.8245e-02,  7.1338e-02, -1.3930e-01,  6.5066e-02,  1.7957e-02,
          -8.5926e-02, -1.7173e-01,  1.8378e-01,  5.7447e-02,  1.1668e-01,
           9.6384e-02,  2.3573e-02,  3.6470e-02, -1.2993e-01,  8.9927e-03,
          -9.6132e-02, -1.1727e-02,  1.2519e-01, -6.4635e-02, -1.1738e-01,
          -2.1501e-02, -1.1976e-02, -6.5760e-03, -2.4115e-01,  5.4326e-02,
          -5.8885e-02, -1.9089e-04, -1.4354e-01, -2.0395e-01, -1.4770e-01,
           9.0752e-02, -1.0371e-01,  2.7519e-02, -8.9220e-02, -1.0204e-01,
          -1.5541e-01, -1.7172e-01,  2.4143e-01,

torch.Size([1, 78, 35])

In [20]:
from diffusion.diffusion import gaussian_diffusion as gd
from diffusion.diffusion.respace import SpacedDiffusion, space_timesteps

def create_gaussian_diffusion(
        diffusion_steps, # number eg 1000
        noise_schedule, # can be 'linear', 'cosine'
        sigma_small, # default True
        lambda_vel, lambda_rcxyz, lambda_fc # for geometric loss, we don't have fc, default 1 for rest
        ):
    # default params
    predict_xstart = True  # we always predict x_start (a.k.a. x0), that's our deal!
    steps = diffusion_steps
    scale_beta = 1.  # no scaling
    timestep_respacing = ''  # can be used for ddim sampling, we don't use it.
    learn_sigma = False
    rescale_timesteps = False

    betas = gd.get_named_beta_schedule(noise_schedule, steps, scale_beta)
    loss_type = gd.LossType.MSE

    if not timestep_respacing:
        timestep_respacing = [steps]

    return SpacedDiffusion(
        use_timesteps=space_timesteps(steps, timestep_respacing),
        betas=betas,
        model_mean_type=(
            gd.ModelMeanType.EPSILON if not predict_xstart else gd.ModelMeanType.START_X
        ),
        model_var_type=(
            (
                gd.ModelVarType.FIXED_LARGE
                if not sigma_small
                else gd.ModelVarType.FIXED_SMALL
            )
            if not learn_sigma
            else gd.ModelVarType.LEARNED_RANGE
        ),
        loss_type=loss_type,
        rescale_timesteps=rescale_timesteps,
        lambda_vel=lambda_vel,
        lambda_rcxyz=lambda_rcxyz,
        lambda_fc=lambda_fc,
    )

In [21]:
model = MotionTransformer(pos_dim=pos_dim, latent_dim=128, ff_size=512, num_layers=4, num_heads=4, dropout=0.1, activation="gelu").to(device)
diffusion = create_gaussian_diffusion(diffusion_steps=1000, noise_schedule="cosine", sigma_small=True, lambda_vel=0.5, lambda_rcxyz=0.5, lambda_fc=1)

In [23]:
class DefaultArgs:
    def __init__(self, save_dir, model_path, eval_model_path):
        # Base options
        self.cuda = True
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.seed = 10
        self.batch_size = 1

        # Diffusion options
        self.noise_schedule = 'cosine'
        self.diffusion_steps = 1000
        self.sigma_small = True

        # Model options
        self.arch = 'trans_enc'
        self.emb_trans_dec = False
        self.layers = 4
        self.latent_dim = 128
        self.cond_mask_prob = 0.1
        self.lambda_rcxyz = 1.0
        self.lambda_vel = 1.0
        self.lambda_fc = 1.0
        self.unconstrained = False  # This is inferred from the 'action' parameter

        # Data options
        self.dataset = 'humanml'
        self.data_dir = ""

        # Training options
        self.save_dir = save_dir
        self.overwrite = False
        self.train_platform_type = 'NoPlatform'
        self.lr = 1e-4
        self.weight_decay = 0.0
        self.lr_anneal_steps = 0
        self.eval_batch_size = 16
        self.eval_split = 'test'
        self.eval_during_training = False
        self.eval_rep_times = 3
        self.eval_num_samples = 1000
        self.log_interval = 1000
        self.save_interval = 4000
        self.num_steps = 20000
        # self.num_frames = 29
        self.resume_checkpoint = ""

        # Sampling options
        self.model_path = model_path
        self.output_dir = ''
        self.num_samples = 10
        self.num_repetitions = 3
        self.guidance_param = 2.5

        # Generate options
        self.motion_length = 6.0
        self.input_text = ''
        self.action_file = ''
        self.text_prompt = ''
        self.action_name = ''

        # Edit options
        self.edit_mode = 'in_between'
        self.text_condition = ''
        self.prefix_end = 0.25
        self.suffix_start = 0.75

        # Evaluation options
        self.eval_model_path = eval_model_path
        self.eval_mode = 'wo_mm'
        self.eval_guidance_param = 2.5


In [24]:
expname = "model-v3-same-pos-data-more-dims-epochs"

args = DefaultArgs(save_dir=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}", model_path=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/model.pt", eval_model_path=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/model.pt")
args.device

'cuda:0'

In [25]:
from train.training_loop import TrainLoop
TrainLoop(args, None , model, diffusion, dataloader).run_loop()

Starting epoch 0/21


  0%|          | 0/1000 [00:00<?, ?it/s]

--------------------------
| grad_norm   | 255      |
| loss        | 239      |
| loss_q0     | 32       |
| loss_q1     | 92.4     |
| loss_q2     | 295      |
| loss_q3     | 519      |
| param_norm  | 37.7     |
| pos_loss    | 239      |
| pos_loss_q0 | 32       |
| pos_loss_q1 | 92.4     |
| pos_loss_q2 | 295      |
| pos_loss_q3 | 519      |
| samples     | 1        |
| step        | 0        |
--------------------------
step[0]: loss[239.48538]
saving model...


  1%|          | 6/1000 [00:00<01:17, 12.78it/s]

Skipping evaluation for now.


...

--------------------------
| grad_norm   | 195      |
| loss        | 214      |
| loss_q0     | 10.4     |
| loss_q1     | 63.4     |
| loss_q2     | 277      |
| loss_q3     | 495      |
| param_norm  | 56.7     |
| pos_loss    | 214      |
| pos_loss_q0 | 10.4     |
| pos_loss_q1 | 63.4     |
| pos_loss_q2 | 277      |
| pos_loss_q3 | 495      |
| samples     | 2e+04    |
| step        | 2e+04    |
--------------------------
step[20000]: loss[214.44123]
saving model...


  0%|          | 4/1000 [00:00<01:13, 13.46it/s]

Skipping evaluation for now.


100%|██████████| 1000/1000 [00:29<00:00, 33.87it/s]

saving model...
Skipping evaluation for now.


In [26]:
dataset[0].shape

torch.Size([78, 35])

In [27]:
num_frames = dataset[0].shape[0]
num_feats = dataset[0].shape[1]

In [28]:
sample_batch_size = 1

all_motions = []
for rep_i in range(args.num_repetitions):
    print(f'### Sampling [repetitions #{rep_i}]')

    sample_fn = diffusion.p_sample_loop

    sample = sample_fn(
        model,
        (sample_batch_size, num_frames, num_feats),
        clip_denoised=False,
        model_kwargs={"y": {}},
        skip_timesteps=0,  # 0 is the default value - i.e. don't skip any step
        init_image=None,
        progress=True,
        dump_steps=None,
        noise=None,
        const_noise=False,
    )

    all_motions.append(sample.cpu().numpy())

### Sampling [repetitions #0]


100%|██████████| 1000/1000 [00:03<00:00, 319.57it/s]


### Sampling [repetitions #1]


100%|██████████| 1000/1000 [00:03<00:00, 281.97it/s]


### Sampling [repetitions #2]


100%|██████████| 1000/1000 [00:03<00:00, 289.82it/s]


In [29]:
all_motions = np.concatenate(all_motions, axis=0)
all_motions.shape

(3, 78, 35)

In [30]:
import numpy as np

def save_motions(all_motions, output_dir):
    for i, motion in enumerate(all_motions):
        filename = f"motion_{i}.npy"
        filepath = os.path.join(output_dir, filename)
        np.save(filepath, motion)
        print(f"Motion {i} saved as {filename}")

save_motions(all_motions, f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/sampled_motions/")

Motion 0 saved as motion_0.npy
Motion 1 saved as motion_1.npy
Motion 2 saved as motion_2.npy
